# OpenAI Assistants API tutorial 

https://www.datacamp.com/tutorial/open-ai-assistants-api-tutorial

**Azure Assistant Reference:**  

Retrieval feature was not available when exploring since it was in a preview release.

https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant

**OpenAI Assistant Reference:**

https://platform.openai.com/docs/assistants/overview

> Note that the below code is modified based on the latest update of the Assistant API from OpenAI

## Import relevant libraries

In [ ]:
!pip install --upgrade openai

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os 
from openai import OpenAI

## Environment configuration

In [ ]:
client = OpenAI()

Test client by asking a sample question.

In [ ]:
message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."},
                {"role": "user", "content": "Who were the founders of Microsoft?"}]

completion = client.chat.completions.create(
  model="gpt-4o",
  messages = message_text,
  temperature=0.7,
  max_tokens=100,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

completion.choices[0].message.content

## Assistant Creation

### [Step 1: Create a new Assistant with File Search Enabled](https://platform.openai.com/docs/assistants/tools/file-search/step-1-create-a-new-assistant-with-file-search-enabled)

In [ ]:
# Create the assistant

assistant = client.beta.assistants.create(
  name="Scientific Paper Assistant",
  instructions="You are a polite and expert knowledge retrieval assistant. Use the documents provided as a knowledge base to answer questions.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

## File Upload

### [Step 2: Upload files and add them to a Vector Store](https://platform.openai.com/docs/assistants/tools/file-search/step-2-upload-files-and-add-them-to-a-vector-store)

Create a vector store object to hold the file.

In [ ]:
# Create a vector store called "OpenAI Assistants Exploration"
vector_store = client.beta.vector_stores.create(name="OpenAI Assistants Exploration")

Prepare the files to upload

In [ ]:
# Ready the files for upload to OpenAI
file_paths = ["data/transformer_paper.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

Upload the files to the vector store as batches

In [ ]:
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

Update the assistant to include the created vector store for retrieval purpose.

### [Step 3: Update the assistant to to use the new Vector Store](https://platform.openai.com/docs/assistants/tools/file-search/step-3-update-the-assistant-to-to-use-the-new-vector-store)

In [ ]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

## Initiate interaction

### [Step 4: Create a thread - with File attachment](https://platform.openai.com/docs/assistants/tools/file-search/step-4-create-a-thread)

You can also attach files as Message attachments on your thread. Doing so will create another vector_store associated with the thread, or, if there is already a vector store attached to this thread, attach the new files to the existing thread vector store. When you create a Run on this thread, the file search tool will query both the vector_store from your assistant and the vector_store on the thread.

In this example, we have attached another pdf named [Intelizign Loan Policy.pdf](./data/Intelizign%20Loan%20Policy.pdf).


In [ ]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("data/Intelizign Loan Policy.pdf", "rb"), purpose="assistants"
)
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "what is the criteria to avail the loan amount?",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

### [Step 4: Create a thread - without File attachment using attached vector store](https://platform.openai.com/docs/assistants/tools/file-search/step-4-create-a-thread)

In [ ]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Why do authors use the self-attention strategy in the paper?"
    }
  ]
)
print(thread)

### [Step 5: Create a run and check the output - Without Streaming](https://platform.openai.com/docs/assistants/tools/file-search/step-5-create-a-run-and-check-the-output)

In [ ]:
from openai import OpenAI
 
client = OpenAI()
 
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
    #temperature=0,
    #top_p=1,
    #max_prompt_tokens=1000, # This can result in a failure if the retrieval cannot fit within the context window.
    #max_completion_tokens=500,
    # Overrides the instructions provided during assistant instantiation.
    #instructions="Please address the user as Jane Doe. The user has a premium account.",
    #additional_instructions="Please address the user as chengaiah",
)

Check run status

In [ ]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

Check Run steps

In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

print(run_steps)

### [Step 5: Create a run and check the output - With Streaming](https://platform.openai.com/docs/assistants/tools/file-search/step-5-create-a-run-and-check-the-output)

In [ ]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI
 
client = OpenAI()
 
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))


# Then, we use the stream SDK helper
# with the EventHandler class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    additional_instructions="Please address the user as chengaiah",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

## Utility actions

### Delete thread

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.beta.threads.delete(thread.id)
print(response)

### Cancel All Runs associated to a thread

In [ ]:
from openai import OpenAI
client = OpenAI()

runs = client.beta.threads.runs.list(
  thread.id
)

for run in runs:
    print(f"Cancelling run with id: {run.id}")
    cancelled_run = client.beta.threads.runs.cancel(
      thread_id=thread.id,
      run_id=run.id
    )
    print(cancelled_run)